In [2]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

# Generate dummy data
import numpy as np
# TODO
# x_train = ...
# y_train = ...
# x_test = ...
# y_test = ...

# bone_length,rotting_flesh,hair_length,has_soul,color
INPUT_DIM = 5
# 'Ghost', 'Goblin', and 'Ghoul'
OUTPUT_DIM = 3

model = Sequential()

# Basic model inspired from:
# https://keras.io/getting-started/sequential-model-guide/
# section: "Multilayer Perceptron (MLP) for multi-class softmax classification"
#
# Play around with:
# * dropout
# * learning algorithm and its parameters
# * activation
# * learning rate
model.add(Dense(10, activation='relu', input_dim=INPUT_DIM))
model.add(Dense(OUTPUT_DIM, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

model.fit(x_train, y_train,
          epochs=20,
          batch_size=128)
score = model.evaluate(x_test, y_test, batch_size=128)

NameError: name 'keras' is not defined